In [1]:
# importing all ther required libraries for building a model and validation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
import warnings

In [2]:
# importing the dataframe for final exam
df = pd.read_csv(r"C:\Users\srika\OneDrive\Documents\York\Sem-2 york\MBAN 6120 - Data Science II\Assignment 1\train.csv",sep=';')

Conduct exploratory analysis on the provided dataset. State your hypothesis based on the insights from your exploration.

In [3]:
df.sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
12404,36,technician,married,secondary,no,283,yes,yes,unknown,27,jun,56,2,-1,0,unknown,no
37032,51,services,single,secondary,no,516,yes,no,cellular,13,may,370,1,363,1,failure,no
35476,27,student,single,secondary,no,8366,no,no,cellular,7,may,458,2,349,1,other,no
38732,32,blue-collar,single,secondary,no,217,yes,no,cellular,15,may,692,3,-1,0,unknown,yes
11720,49,technician,single,unknown,no,443,no,no,unknown,20,jun,25,11,-1,0,unknown,no


In [4]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [5]:
dict = {'yes':1,'no':0}
df['default'] = df['default'].map(dict)
df['housing'] = df['housing'].map(dict)
df['loan'] = df['loan'].map(dict)
df['y'] = df['y'].map(dict)
df['marital'].unique()

array(['married', 'single', 'divorced'], dtype=object)

In [6]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,45211.0,NaN,NaN,NaN,40.93621,10.618762,18.0,33.0,39.0,48.0,95.0
job,45211,12,blue-collar,9732,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital,45211,3,married,27214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,45211,4,secondary,23202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default,45211.0,NaN,NaN,NaN,0.018027,0.133049,0.0,0.0,0.0,0.0,1.0
balance,45211.0,NaN,NaN,NaN,1362.272058,3044.765829,-8019.0,72.0,448.0,1428.0,102127.0
housing,45211.0,NaN,NaN,NaN,0.555838,0.496878,0.0,0.0,1.0,1.0,1.0
loan,45211.0,NaN,NaN,NaN,0.160226,0.36682,0.0,0.0,0.0,0.0,1.0
contact,45211,3,cellular,29285,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,45211.0,NaN,NaN,NaN,15.806419,8.322476,1.0,8.0,16.0,21.0,31.0


from the above : all have medium hence they are all integers but need to check if the continuous or catagorical<br>
'age' mean and mediun and almost in the same place.<br>
'ana'- is binary<br>
 'c_pho' - max value is too high<br>
 'diab' - is binary<br>
 'e_fract' - mean and median are close together<br>
 'hbp' - is binary<br>
 'plat' - need to analyse more<br>
 's_c' - mean and median are normally distributed<br>
 's_s' - mean and median are normally distributed<br>
 'sex' - is binary male / female<br>
 'smok' - is binary yes/no<br>
 'time' - time of death<br>
 'DEATH' - our target variable is binary which is good<br>
<br>
can you see outliers ? what do you understand from 50% and mean.<br>
list variables which are catagorical.<br>
<br>
which variables are continuous.<br>

can you see outliers ? what do you understand from 50% and mean.
list variables which are catagorical.

which variables are continuous.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  int64 
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  int64 
 7   loan       45211 non-null  int64 
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  int64 
dtypes: int64(11), object(6)
memory usage: 5.9+ MB


missing values:

we can see that the total data is around 1050 in Sate date but every other value is missing few details.

Total count of null are less than 10% hence droping the Values

In [8]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [10]:
df.drop_duplicates(inplace=True)

Prepare data for modelling (i.e., data cleaning and feature engineering).

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  int64 
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  int64 
 7   loan       45211 non-null  int64 
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  int64 
dtypes: int64(11), object(6)
memory usage: 5.9+ MB


making dummies

In [ ]:
job = pd.get_dummies(df['job'],dtype=int)
pd.concat([df,sex],axis=1)
df.drop(['sex'],axis=1,inplace=True)

In [ ]:
df['Remarks'] = df['Remarks'].astype('category')
df['Gen_new'] = df['Gender'].cat.codes
enc = OneHotEncoder()
enc_data = pd.DataFrame(enc.fit_transform(df[['Gen_new', 'Rem_new']]).toarray())
df = df.join(enc_data)


In [ ]:
df['target'].unique()

In [ ]:
df['target'] = df['target'].map({'Abnormal':1, 'Normal':0})

In [ ]:
# univariate analysis - first we check the output variable and make sure it's balanced
df.groupby('target').count()

In [ ]:
# ratio is
60*100 / (122+60)

thats decently balanced dataset. so planning to skip smote

In [ ]:
Y = Y.astype('int')
X_balance,Y_balance = SMOTE().fit_sample(X,Y)
X_balance = pd.DataFrame(X_balance, columns = X.columns)

In [ ]:
sns.pairplot(df,hue = 'target')

In [ ]:
df.columns

In [ ]:
df.nunique()

In [ ]:
df_sex = df.groupby(['DEATH_EVENT','sex'])[['smoking']].count().unstack()
df_sex['total'] = df_sex['smoking'][0] + df_sex['smoking'][1]
df_sex['percent_0'] = df_sex['smoking'][0] *100/ df_sex['total']
df_sex['percent_1'] = df_sex['smoking'][1] *100/ df_sex['total']
df_sex

In [ ]:
df.groupby(['diab', 'y']).size().unstack() / df.groupby('y').size()

no difference

In [ ]:
df.groupby(['ana','DEATH']).size().unstack() / df.groupby('DEATH').size()

ver little difference in values

change of data type using astype

In [ ]:
sns.histplot(df['s_c'])

In [ ]:
df.drop(df[df['s_c']>8].index, axis=0,inplace=True)

log transform only to make it more normally distributed 

In [ ]:
df['s_c']=np.log(df['s_c'])

In [ ]:
sns.histplot(df['s_c'])

for continuous variables: plot the boxplots with the x-axis as the output variable, and y-axis as the continuous variable 

In [ ]:
df.nunique()

In [ ]:
df.columns

In [ ]:
fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(15, 5))
sns.boxplot(x='target', y='v1', data=df, order=df.groupby('target')['v1'].median().sort_values().index, ax=ax1)
sns.boxplot(x='target', y='v1', data=df, order=df.groupby('target')['v1'].median().sort_values().index, ax=ax2)
sns.boxplot(x='target', y='v1', data=df, order=df.groupby('target')['v1'].median().sort_values().index, ax=ax3)
plt.show()

'age',  'c_pho', 'e_fract' - show signs wiht different death<br>
'ana','plat','s_c', 's_s', 'time' - show signs wiht different death <br>

above : are the variables meaning full for analysis

catagorical variable : 

In [ ]:
df.groupby(['diab','DEATH']).size().unstack()

In [ ]:
df.groupby(['diab', 'DEATH']).size().unstack() / df.groupby('DEATH').size()

above : do they show difference in the output ?

In [ ]:
df.describe(include='all').transpose()

In [ ]:
sns.histplot(df['Parch'])
plt.show()
sns.histplot(df['SibSp'])
plt.show()
sns.histplot(df['Fare'])
plt.show()

In [ ]:
df.drop(df[df['c_pho'] > 3000].index, axis=0, inplace=True)

In [ ]:
df['var'] = np.log(df['var'])

In [ ]:
# check with PCA to drop features.
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
df_std = scaler.fit_transform(df)

# Perform PCA
pca = PCA()
df_pca = pca.fit_transform(df_std)

# The transformed data is an array, convert it back into a dataframe
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(len(df.columns))])

# Print the cumulative explained variance ratio
cumsum_variance = np.cumsum(pca.explained_variance_ratio_)
print('Cumulative explained variance ratio:', cumsum_variance)

'00'number of varaibles explain the target value by 90%

feature selection 

In [ ]:
dc = df.corr()
dc['target'].sort_values(ascending=False).index

In [ ]:
 
df=df[['target', 'sqft_living', 'grade', 'sqft_living15', 'bathrooms', 'view',
       'bedrooms', 'lat', 'waterfront', 'floors', 'yr_renovated', 'renovated',
       'yr_built', 'condition', 'long', 'year', 'quarter', 'month', 'age']]
plt.figure(figsize=(30, 15))
sns.heatmap(df.corr(),annot=True,cmap='viridis', square=True)

In [ ]:
dc.drop('DEATH',axis=1,inplace=True)
dc.drop('DEATH',axis=0,inplace=True)
dc.replace(1, 0, inplace=True)

In [ ]:
row_index, col_index = np.where(dc.values == dc.max().max())
print(dc.index[row_index[0]])
print(dc.columns[col_index[0]])

In [ ]:
plt.figure(figsize=(5, 5))
sns.scatterplot(data=df,x ='time',y ='s_c',hue='DEATH')
plt.show()

In [ ]:
# drop columns based on PCA count. 
df.drop('DEATH',axis=1,inplace=True)

some of the featueres are well corellared with each other and one of them can be selected to represent the model.


In [ ]:
df['hd_cons'] = df['hd'].map({0:0, 1:1, 2:1, 3:1, 4:1})

binning the continuous variable:

In [ ]:
def categorize_Age(Age):
    if Age > 60:
        return "High"
    elif 40 <= Age <= 60:
        return "Medium 1"
    elif 20 <= Age <= 40:
        return "Medium_2"
    else:
        return "Low"

df['Age_cat'] = df['Age'].apply(categorize_Age)


define new metrics (e.g., multiply columns together or create custom categories based on multiple variables) 

In [ ]:
df['var'] = df['var1']+df['var2']
df['var'].diff(1)
df['var']=df['var']-1

 adding dummies for all catorical varianbles: 

In [ ]:
cat_columns = ['AGE', 'YEARS_EMPLOYED', 'INCOME', 'INCOME_PER_MEMBER', 'INCOME_PER_CHILD']

In [ ]:
df = pd.get_dummies(df, columns=['Age_cat'], prefix='Age',dtype=int)
X_encoded = pd.get_dummies(X, columns=cat_columns, dtype=int)

In [ ]:
df = pd.concat([df[num_columns],X_encoded], axis=1)

In [ ]:
df.sample(5)

outlier verification : 

end o f data preprocessing 

-------------------------------------

setting up the model

In [ ]:
df.columns()

In [ ]:
x=df.drop('target', axis=1)
y=df['Survived']


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

standardizaton & normalization :scaling

In [ ]:
# Transform the train and test data
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


# num_columns = ['AGE', 'YEARS_EMPLOYED', 'INCOME', 'INCOME_PER_MEMBER', 'INCOME_PER_CHILD']
# scaler = StandardScaler()
# scaler.fit_transform(df[])

# norm = MinMaxScalar()
# norm.fit_transform(df[])

In [ ]:
x_train.shape

In [ ]:
x_test.shape

both have 6 columns each so the shape is good.

In [ ]:
plt.figure(figsize=(30, 15))
sns.scatterplot(x = x_train[:,0],y = x_train[:,1],hue=y_train,style = y_train)
plt.show()
# Slicing  - all rows, first two high correlated feature columns 
# for looking at y - spread. 

the train data doesnt show any great clustering

-------------------------------------

### modelling 

Build a model to predict the output variable. Hint: you will need to try several models to find the best model for your problem.


Logistic Regression, Naive Bayes, KNN, SVM, Decision Tree. 

In [ ]:
# Model Selection
log = LogisticRegression()
#
nb = GaussianNB()
#
knn = KNeighborsClassifier(n_neighbors=7)
#
tree = DecisionTreeClassifier(random_state=123)
#
svc = SVC(kernel='linear', random_state=123,probability=True)
#
models = [log, nb, knn, tree, svc]

In [ ]:
warnings.filterwarnings("ignore")

model training

In [ ]:
log.fit(x_train,y_train)
nb.fit(x_train,y_train)
knn.fit(x_train,y_train)
tree.fit(x_train,y_train)
svc.fit(x_train,y_train)

Cross validation 

In [ ]:
cv_scores = cross_val_score(log, x_train, y_train, cv=10)
print(cv_scores)
print(cv_scores.mean())


In [ ]:
cv_scores = cross_val_score(nb, x_train, y_train, cv=10)
print(cv_scores)
print(cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(knn, x_train, y_train, cv=10)
print(cv_scores)
print(cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(tree,  x_train, y_train, cv=10)
print(cv_scores)
print(cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(svc, x_train, y_train, cv=10)
print(cv_scores)
print(cv_scores.mean())

In [ ]:
yhat_log = log.predict_proba(x_test)
yhat_nb = nb.predict_proba(x_test)
yhat_knn = knn.predict_proba(x_test)
yhat_tree = tree.predict_proba(x_test)
yhat_svc = svc.predict_proba(x_test)

In [ ]:
y_pred_log = log.predict(x_test)
y_pred_nb = nb.predict(x_test)
y_pred_knn = knn.predict(x_test)
y_pred_tree = tree.predict(x_test)
y_pred_svc = svc.predict(x_test)

In [ ]:
fpr_log, tpr_log, thresholds_l = roc_curve(y_test, yhat_log[:,1])
fpr_nb, tpr_nb, thresholds_nb = roc_curve(y_test, yhat_nb[:,1])
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, yhat_nb[:,1]) # * check again
fpr_tree, tpr_tree, thresholds_tree = roc_curve(y_test, yhat_tree[:,1])
fpr_svc, tpr_svc, thresholds_svc = roc_curve(y_test, yhat_svc[:,1])

In [ ]:
metrics = pd.DataFrame(index = ['accuracy','precision', 'recall'], columns =['logisticReg','NaiveBayes','KNN','Tree','svc']   )

In [ ]:
metrics.loc['accuracy','logisticReg'] = accuracy_score(y_pred=y_pred_log,y_true=y_test)
metrics.loc['precision','logisticReg'] = precision_score(y_pred=y_pred_log,y_true=y_test)
metrics.loc['recall','logisticReg'] = recall_score(y_pred=y_pred_log,y_true=y_test)
print(confusion_matrix(y_pred=y_pred_log,y_true=y_test))
print("Classification Report:")
print(classification_report(y_test,y_pred_log))

In [ ]:
metrics.loc['accuracy','NaiveBayes'] = accuracy_score(y_pred=y_pred_nb,y_true=y_test)
metrics.loc['precision','NaiveBayes'] = precision_score(y_pred=y_pred_nb,y_true=y_test)
metrics.loc['recall','NaiveBayes'] = recall_score(y_pred=y_pred_nb,y_true=y_test)
print(confusion_matrix(y_pred=y_pred_nb,y_true=y_test))
print("Classification Report:")
print(classification_report(y_test,y_pred_nb))

In [ ]:
metrics.loc['accuracy','KNN'] = accuracy_score(y_pred=y_pred_knn,y_true=y_test)
metrics.loc['precision','KNN'] = precision_score(y_pred=y_pred_knn,y_true=y_test)
metrics.loc['recall','KNN'] = recall_score(y_pred=y_pred_knn,y_true=y_test)
print(confusion_matrix(y_pred=y_pred_knn,y_true=y_test))
print("Classification Report:")
print(classification_report(y_test,y_pred_knn))
# order of detail inputed is important. Since predictied and actual order can interchange the values of FP & FN.

In [ ]:
metrics.loc['accuracy','Tree'] = accuracy_score(y_pred=y_pred_tree,y_true=y_test)
metrics.loc['precision','Tree'] = precision_score(y_pred=y_pred_tree,y_true=y_test)
metrics.loc['recall','Tree'] = recall_score(y_pred=y_pred_tree,y_true=y_test)
print(confusion_matrix(y_pred=y_pred_tree,y_true=y_test))
print("Classification Report:")
print(classification_report(y_test,y_pred_tree))

In [ ]:
metrics.loc['accuracy','svc'] = accuracy_score(y_pred=y_pred_svc,y_true=y_test)
metrics.loc['precision','svc'] = precision_score(y_pred=y_pred_svc,y_true=y_test)
metrics.loc['recall','svc'] = recall_score(y_pred=y_pred_svc,y_true=y_test)
print(confusion_matrix(y_pred=y_pred_svc,y_true=y_test))
print("Classification Report:")
print(classification_report(y_test,y_pred_svc))

Evaluate each model using the evaluation techniques you have learned in class.


In [ ]:
# plotting decision tree
from sklearn.tree import plot_tree
plt.figure(figsize=(20,20))
feature_names = x_train.columns
plot_tree(tree, filled=True, rounded=True, feature_names=feature_names, class_names=['0', '1'])
plt.show()

some of the features have been adjusted based on decision tree outputs.

In [ ]:
100*metrics

ROC curve overlap

In [ ]:
plt.figure(figsize=(30, 30))
plt.plot(fpr_log,tpr_log, label='log')
plt.plot(fpr_nb,tpr_nb, label='nb')
plt.plot(fpr_knn,tpr_knn, label='knn')
plt.plot(fpr_tree,tpr_tree, label='tree')
plt.plot(fpr_svc,tpr_svc, label='svc')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.legend()

In [ ]:
# auc scores
print('logistic_regression : ',auc(fpr_log,tpr_log))
print('NaiveBayes : ',auc(fpr_nb,tpr_nb))
print('KNN : ',auc(fpr_knn,tpr_knn))
print('Tree : ',auc(fpr_tree,tpr_tree))
print('Svc : ',auc(fpr_svc,tpr_svc))

----------------- is best . Doing hyper parameter tuning based off it.

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 10],'solver': ['lbfgs', 'liblinear', 'sag', 'saga']}

model = LogisticRegression()
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')
randomsearch = RandomizedSearchCV(model, param_grid, cv=10, n_iter=100, scoring='accuracy')

grid_search.fit(x_train, y_train)
randomsearch.fit(x_train, y_train)
print("Best Hyperparameters grid:", grid_search.best_params_)
print("Best Hyperparameters Random:", randomsearch.best_params_)

In [ ]:
# Hyperparameter grids for each model
log_reg_param_grid = {'C': [0.01, 0.1, 1, 10],'solver': ['lbfgs', 'liblinear', 'sag', 'saga']}
knn_param_grid = {'n_neighbors': list(range(1, 20)),'weights': ['uniform', 'distance'],'metric': ['l1', 'euclidean', 'manhattan', 'sokalsneath', 'hamming', 'sokalmichener', 'rogerstanimoto', 'braycurtis', 'dice', 'jaccard', 'pyfunc', 'haversine', 'matching', 'cityblock', 'correlation', 'l2', 'nan_euclidean', 'chebyshev', 'p', 'wminkowski', 'minkowski', 'cosine', 'infinity', 'russellrao', 'seuclidean', 'canberra', 'kulsinski', 'sqeuclidean', 'yule', 'mahalanobis']}
decision_tree_param_grid = {'criterion': ['gini', 'entropy'],'max_depth': [None, 10, 20, 30],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]}
svc_param_grid = {'C': [0.1, 1, 10],'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],'gamma': ['scale', 'auto'],'class_weight': [None, 'balanced']}

# Create models
log_reg_model = LogisticRegression()
naive_bayes_model = GaussianNB()
knn_model = KNeighborsClassifier()
decision_tree_model = DecisionTreeClassifier()
svc_model = SVC()

# Perform RandomizedSearchCV  for each model
random_search_log_reg = RandomizedSearchCV(log_reg_model, log_reg_param_grid, cv=10, n_iter=100, scoring='accuracy')
random_search_knn = RandomizedSearchCV(knn_model, knn_param_grid, cv=10, n_iter=100, scoring='accuracy')
random_search_decision_tree = RandomizedSearchCV(decision_tree_model, decision_tree_param_grid, cv=10, n_iter=100, scoring='accuracy')
random_search_svc = RandomizedSearchCV(svc_model, svc_param_grid, cv=10, n_iter=100, scoring='accuracy')


# Perform GridSearchCV for each model
grid_search_log_reg = GridSearchCV(log, log_reg_param_grid, cv=10, scoring='accuracy')
grid_search_knn = GridSearchCV(knn_model, knn_param_grid, cv=10, scoring='accuracy')
grid_search_decision_tree = GridSearchCV(decision_tree_model, decision_tree_param_grid, cv=10, scoring='accuracy')
grid_search_svc = GridSearchCV(svc_model, svc_param_grid, cv=10, scoring='accuracy')


# Fit the models and find best hyperparameters
random_search_log_reg.fit(x_train, y_train)
random_search_knn.fit(x_train, y_train)
random_search_decision_tree.fit(x_train, y_train)
random_search_svc.fit(x_train, y_train)

print("Best parameters from RandomSearch: ", randomsearch_knn.best_params_)

from both considering random: <br>
{'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30, 'criterion': 'entropy'}<br>

applying the best Hyperparameters:Modle optimization

In [ ]:
best_model = LogisticRegression(**randomsearch.best_params_)
cv_scores = cross_val_score(best_model, x_train, y_train, cv=10, scoring='accuracy')
best_model.fit(x_train, y_train)
# Predict
y_hat_best = best_model.predict_proba(x_test)
y_pred_best = best_model.predict(x_test)

add some details of the output


* Check performance metrics: precision, recall, F1 score, roc_auc, accuracy score 
* If your model isn't performing well, try adding/removing variables, or engineering more features 
* Once you're confident you have tried everything possible to build the best possible model, you can finalize your model 

In [ ]:
print(confusion_matrix(y_pred=y_pred_best, y_true=y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_best))

7. Explaining your results 
* Explain your results by always connecting your decisions to the business problem provided. 
* Explain your process > how did you decide to clean the data? how do you know that was the best decision? what did you learn from exploratory analysis, and how did this inform your modelling decisions? Which features did you engineer? how did you pick these?  
* Explain the final model > which variables were included in the model, and what was the model performance? 
* Explain how this model can be applied to the future to predict outcomes > how will the business use the model? 

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, best_model.predict_proba(x_test)[:, 1])
plt.plot(fpr, tpr, label=best_model.__class__.__name__)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

e. Explain your model to a business executive. Your explanation should include the following:

i. Your data exploration findings, initial hypothesis, and whether the hypothesis proved true.

ii. Your approach for building and evaluating the model. 

iii. The final model you selected and how you know this is the best model for the problem.

iv. How the model should be used by the business, in business process.

did you refer notes ? 


fir , trankssform, fit transform.
closing a model prediction. how to finsih the mdoel.
model deplyment.
confirmatory data analysis ?